In [2]:
import pandas as pd
#df=pd.read_csv('https://cloud.minsa.gob.pe/s/xJ2LQ3QyRW38Pe5/download', sep=';', usecols=['FECHA_CORTE', 'EDAD_DECLARADA', 'SEXO', 'FECHA_FALLECIMIENTO', 'DEPARTAMENTO'], parse_dates=['FECHA_FALLECIMIENTO'])
df=pd.read_csv('fallecidos_covid.csv', sep=';', usecols=['FECHA_CORTE', 'EDAD_DECLARADA', 'SEXO', 'FECHA_FALLECIMIENTO', 'DEPARTAMENTO'], parse_dates=['FECHA_FALLECIMIENTO'])
df=df[(df['SEXO']=='FEMENINO') | (df['SEXO']=='MASCULINO')]
fecha_corte=df['FECHA_CORTE'].drop_duplicates().set_axis(['fecha_corte'])
#fecha_corte.to_json("../resultados/fecha_corte_negativos.json")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf3 in position 199: invalid continuation byte

In [ ]:
# DIARIO FALLECIDOS
df_fallecidos=df[['FECHA_FALLECIMIENTO','SEXO', 'EDAD_DECLARADA']].groupby(['FECHA_FALLECIMIENTO', 'SEXO']).count()
df_fallecidos=df_fallecidos.reset_index()
df_fallecidos=df_fallecidos.pivot(index='FECHA_FALLECIMIENTO', columns='SEXO', values='EDAD_DECLARADA')
df_fallecidos=df_fallecidos.rename_axis(None, axis=1)
df_fallecidos=df_fallecidos.fillna(0).astype('int')
df_fallecidos

,FEMENINO,MASCULINO
FECHA_FALLECIMIENTO,,
2020-03-03,0,1
2020-03-07,1,0
2020-03-11,1,0
2020-03-14,0,2
2020-03-16,2,2
...,...,...
2021-06-01,152,256
2021-06-02,134,242
2021-06-03,157,210


In [ ]:
# ACUMULADO FALLECIDOS
df_fallecidos_cum=df_fallecidos.cumsum()
df_fallecidos_cum

,FEMENINO,MASCULINO
FECHA_FALLECIMIENTO,,
2020-03-03,0,1
2020-03-07,1,1
2020-03-11,2,1
2020-03-14,2,3
2020-03-16,4,5
...,...,...
2021-06-01,67058,118186
2021-06-02,67192,118428
2021-06-03,67349,118638


In [ ]:
col_poblacion=[426806,
1180638,
430736,
1497438,
668213,
1453711,
1129854,
1357075,
365317,
760267,
975182,
1361467,
2016771,
1310785,
10628470,
1027559,
173811,
192740,
271904,
2047954,
1237997,
899648,
370974,
251521,
589110]

In [ ]:
# ACUMULADO POR DEPARTAMENTO
df_fallecidos_departamento=df[['DEPARTAMENTO','SEXO']]
def function(valueX):
  if 'LIMA' in valueX:
    return 'LIMA'
  else:
    return valueX
df_fallecidos_departamento['DEPARTAMENTO']=df_fallecidos_departamento['DEPARTAMENTO'].map(function)
df_fallecidos_departamento=df_fallecidos_departamento.groupby(['DEPARTAMENTO']).count()
df_fallecidos_departamento['POBLACION']=col_poblacion
df_fallecidos_departamento['INDICE']=round(df_fallecidos_departamento['SEXO']/(df_fallecidos_departamento['POBLACION']/100000)).astype('int')
df_fallecidos_departamento

<ipython-input-5-a04e5e81a75b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fallecidos_departamento['DEPARTAMENTO']=df_fallecidos_departamento['DEPARTAMENTO'].map(function)


,SEXO,POBLACION,INDICE
DEPARTAMENTO,,,
AMAZONAS,1110,426806,3
ANCASH,6265,1180638,5
APURIMAC,1364,430736,3
AREQUIPA,7746,1497438,5
AYACUCHO,1896,668213,3
CAJAMARCA,3777,1453711,3
CALLAO,9583,1129854,8
CUSCO,4238,1357075,3
HUANCAVELICA,1075,365317,3


In [ ]:
# ACUMULADO POR GRUPO ETARIO
bins = [18,20,30,40,50,60,70,80,df['EDAD_DECLARADA'].max()+1]
labels = ['18 a 19 años','20 a 29 años','30 a 39 años','40 a 49 años','50 a 59 años','60 a 69 años','70 a 79 años','80 años a más']
poblacion_por_grupo_etario = [900000,5700000,5300000,4400000,3300000,2300000,1300000,700000] 
df_fallecidos_edades = df
df_fallecidos_edades['GRUPO_ETARIO'] = pd.cut(df['EDAD_DECLARADA'], bins=bins, labels=labels, right=False)
df_fallecidos_edades = df_fallecidos_edades.groupby(['GRUPO_ETARIO'])["SEXO"].count().reset_index()
df_fallecidos_edades.rename(columns = {'SEXO':'FALLECIDOS'}, inplace = True)
df_fallecidos_edades['POBLACION']=poblacion_por_grupo_etario
df_fallecidos_edades['PORCENTAJE']=round(df_fallecidos_edades['FALLECIDOS']/df_fallecidos_edades['POBLACION']*100,2)
df_fallecidos_edades=df_fallecidos_edades.set_index('GRUPO_ETARIO')
df_fallecidos_edades

,FALLECIDOS,POBLACION,PORCENTAJE
GRUPO_ETARIO,,,
18 a 19 años,160,900000,0.02
20 a 29 años,2014,5700000,0.04
30 a 39 años,6161,5300000,0.12
40 a 49 años,15487,4400000,0.35
50 a 59 años,31406,3300000,0.95
60 a 69 años,47996,2300000,2.09
70 a 79 años,45409,1300000,3.49
80 años a más,36866,700000,5.27


In [ ]:
#TOTAL FALLECIDOS
#fallecidos = pd.Series([df['FECHA_CORTE'].unique()[0], df[['FECHA_CORTE'][0]].count(), df_fallecidos.tail(1).sum(axis=1)[0]], index=['fecha_corte', 'total_fallecidos', 'incremento_fallecidos'])
#fallecidos.to_json('../resultados/fallecidos.json')

In [ ]:
# DIARIO FALLECIDOS: TACNA
df_tacna=df[df['DEPARTAMENTO'] == 'TACNA']
df_fallecidos_tacna=df_tacna[['FECHA_FALLECIMIENTO','SEXO', 'EDAD_DECLARADA']].groupby(['FECHA_FALLECIMIENTO', 'SEXO']).count()
df_fallecidos_tacna=df_fallecidos_tacna.reset_index()
df_fallecidos_tacna=df_fallecidos_tacna.pivot(index='FECHA_FALLECIMIENTO', columns='SEXO', values='EDAD_DECLARADA')
df_fallecidos_tacna=df_fallecidos_tacna.rename_axis(None, axis=1)
df_fallecidos_tacna=df_fallecidos_tacna.fillna(0).astype('int')
df_fallecidos_tacna